# Uso de RDD's en Spark

## Creación dun RDD

O primeiro paso é obter o obxecto **SparkContext**.  
Como recomendación persoal, é preferible facelo a partir do obxecto **SparkSession**, xa que este centraliza a configuración e o acceso ás distintas APIs de Spark.


In [2]:
# IMPORTANTE:
# 1. Se se executa en CESGA, ignorar esta cela.

from pyspark.sql import SparkSession

# 2. Se se emprega o clúster adbgonzalez/spark-cluster:
spark = (
    SparkSession
        .builder
        .appName("01-rdd1")
        .getOrCreate()
)



# 3. Se se executa empregando all-spark-notebook,
# comentar o bloque anterior e descomentar a seguinte liña:
# spark = SparkSession.builder.getOrCreate()

# Obter o SparkContext a partir da SparkSession
sc = spark.sparkContext

print(spark.version)


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ea2a708f-bbc0-4c78-86ec-72e51838809f;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 758ms :: artifacts dl 32ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

3.5.7


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60158)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 271, in accum_updates
    num_updates = read_int(self.rf

### A partir de una colección local
Para crear un RDD a partir dunha colección úsase o método parallelize. A continuación amósanse dous exemplos: 
- Creación dun rdd de enteiros.
- Creación dun RDD de strings.

In [3]:
# RDD de enteiros
rdd_int = sc.parallelize(range(10))

# RDD de strings
rdd_st = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split())

# usamos collect para amosar o contido
print(f"rdd de enteiros: {rdd_int.collect()}")
print(f"rdd de strings: {rdd_st.collect()}")

[Stage 0:=============================>                             (1 + 1) / 2]

rdd de enteiros: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rdd de strings: ['Big', 'Data', 'aplicado.', 'Curso', 'de', 'especialización', 'de', 'Inteligencia', 'Artificial', 'y', 'Big', 'Data']


Tamén se pode empregar algún parámetro opcional, como o número de particións.

In [4]:
# RDD de enteiros con 4 particións
rdd_int_par = sc.parallelize(range(20),4)

##RDD de strings con 5 particións
rdd_st_par = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split(),5)


### A partir de fontes de datos

Para iso dispoñemos de dúas opcións:
- **textFile**: para crear un RDD a partir dun ficheiro de texto
- **wholeTextFiles**: para crear un RDD a partir de varios ficheiros de texto


In [5]:
# RDD a partir dun arquivo
rdd_file = sc.textFile("/data/flight-data/csv/2015-summary.csv")

# Amosamos 5 primeiros elementos con take (verase máis adiante)
print(rdd_file.take(5))

# RDD a partir de varios arquivos
rdd_whole_files = sc.wholeTextFiles("/data/flight-data/csv")

# Cada elemento é un arquivo. Está comentado para non saturar o caderno
#rdd_whole_files.take(5)

[Stage 2:>                                                          (0 + 1) / 1]

['DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count', 'United States,Romania,15', 'United States,Croatia,1', 'United States,Ireland,344', 'Egypt,United States,15']


### A partir de DataFrames existentes

Unha forma moi sinxela de crear **RDDs** é a partir dun **DataFrame** ou **Dataset** existente (veranse en sesións posteriores).


In [ ]:
spark.range(10).rdd.collect()

Deste xeito obtemos un **RDD** formado por obxectos de tipo *Row*.  
Para poder manexar estes datos é necesario converter os obxectos de tipo *Row* ao tipo axeitado ou extraer os seus valores, tal e como se amosa no seguinte exemplo:


In [ ]:
spark.range(10).toDF("id").rdd.map(lambda row: row[0]).collect()

## Acciones
Chamamos accións ás operacións sobre rdd que dan como resultado un tipo de datos distinto a RDD (enteiro, colección, etc.). Adoitan ser a última operación que se aplica. Aquí temos as principais:
### collect
Devolve unha `list` con todos os elementos dun RDD. Fai que se executen todas as transformacións previas.


In [9]:
# Elmentos do RDD de enteiros
print (rdd_int.collect())

# Elementos do rdd de strings
print (rdd_st.collect())

# Elementos do rdd de arquivo
#print (rdd_file.collect())

# Elementos do rdd de arquivo
#print (rdd_whole_files.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
<class 'list'>
['Big', 'Data', 'aplicado.', 'Curso', 'de', 'especialización', 'de', 'Inteligencia', 'Artificial', 'y', 'Big', 'Data']


### take
Permite obtener unha `list` cun número determinado de elmentos do RDD.

In [11]:
# Tres primeiros elementos do RDD de enteiros
print (rdd_int.take(3))

# Tres primeiros elementos do RDD de strings
print (rdd_st.take(3))

# Dous primeiros elementos do RDD de arquivo
#print (rdd_file.take(2))

# Dous primeiros elementos do RDD de arquivos
#print (rdd_whole_files.take(2))

[0, 1, 2]
['Big', 'Data', 'aplicado.']


### count
Devolve o número (`int`) de elementos dun RDD.

In [13]:
# Amosamos o número de elementos dos RDD creados ata agora
print ("RDD de enteiros:",rdd_int.count())
print ("RDD de strings: ",rdd_st.count())
print ("RDD dun arquivo: ",rdd_file.count())
print ("RDD de varios arquivos: ",rdd_whole_files.count())

RDD de enteiros: 10
RDD de strings:  12
RDD dun arquivo:  257
RDD de varios arquivos:  6


### reduce

Permite, mediante unha función especificada polo programador, **reducir un RDD a un único valor**.  
Esta función recibe dous parámetros e devolve un único resultado. Tamén se poden empregar funcións *lambda*.



In [15]:
# Exemplo 1, aplicamos a función reduce sumando todos os elementos do RDD de enteiros mediante o uso de lambda
print("Suma total: ",rdd_int.reduce (lambda x,y: x+y))
# Exemplo 2
# Definimos función que devolve a palabra de maior lonxitude.
def word_length_reducer(word1,word2):
    if (len(word1) > len (word2)):
        return word1
    else:
        return word2

# Executamos a función reduce aplicando a función previamente definida.
print ("Palabra máis longa: ",rdd_st.reduce (word_length_reducer))


Suma total:  45
Palabra máis longa:  especialización


### first
Devolve o primeiro elemento dun RDD.

In [17]:
# Primeiro elemento de cada RDD
print ("RDD de enteiros: ",rdd_int.first())
print ("RDD de strings: ",rdd_st.first())


RDD de enteiros:  0
RDD de strings:  Big


### max/min

Devuelve o valor máximo/mínimo dun RDD



In [18]:
# Máximo/mínimo de cada RDD
print("Mínimo do RDD de enteiros: ",rdd_int.min())

print("Máximo do RDD de strings: ", rdd_st.max())

Mínimo do RDD de enteiros:  0
Máximo do RDD de strings:  y


## Transformacións
Enténdese por transformacións aquelas operacións realizadas sobre RDD's que dan como resultado outro RDD.

### map
Permite aplicar unha función especificada polo programador a cada un dos elementos do **RDD**, devolvendo un **RDD do mesmo tamaño** ca o orixinal.


In [20]:
# Exemplo: multiplicamos por 2 cada elemento do RDD de enteiros
print("RDD de enteiros pares: ",rdd_int.map (lambda x: 2*x).collect())

# Exemplo: convertimos cada elemento do RDD de palabras nunha lista:
print ("RDD de listas de caracteres: ",rdd_st.map (lambda x: list(x)).collect())

RDD de enteiros pares:  [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
RDD de listas de caracteres:  [['B', 'i', 'g'], ['D', 'a', 't', 'a'], ['a', 'p', 'l', 'i', 'c', 'a', 'd', 'o', '.'], ['C', 'u', 'r', 's', 'o'], ['d', 'e'], ['e', 's', 'p', 'e', 'c', 'i', 'a', 'l', 'i', 'z', 'a', 'c', 'i', 'ó', 'n'], ['d', 'e'], ['I', 'n', 't', 'e', 'l', 'i', 'g', 'e', 'n', 'c', 'i', 'a'], ['A', 'r', 't', 'i', 'f', 'i', 'c', 'i', 'a', 'l'], ['y'], ['B', 'i', 'g'], ['D', 'a', 't', 'a']]


### flatMap
Permite realizar operacións de tipo *map* que **non son 1:1**.  
A diferenza de `map`, cada elemento de entrada pode xerar **cero, un ou varios elementos de saída**.

Deste xeito, o tamaño do RDD resultante **non ten por que coincidir** co do RDD orixinal.

Por exemplo, no seguinte código aplícase a mesma función ca no caso anterior, pero o resultado é moi distinto, xa que cada elemento pode producir múltiples valores.


In [22]:
# Exemplo: Primeiros 10 caracteres do RDD de palabras
print("Primeiros 10 caracteres do RDD de palabras: ",rdd_st.flatMap(lambda x: list(x)).take(10))

Primeiros 10 caracteres do RDD de palabras:  ['B', 'i', 'g', 'D', 'a', 't', 'a', 'a', 'p', 'l']


### distinct
Elimina os duplicados

In [23]:
# Exemplo, RDD de palabras sen duplicados
print("RDD de palabras sen duplicados: ",rdd_st.distinct().collect())

[Stage 42:>                                                         (0 + 2) / 2]

RDD de palabras sen duplicados:  ['Artificial', 'y', 'aplicado.', 'Big', 'Data', 'Curso', 'de', 'especialización', 'Inteligencia']


### filter
Permite seleccionar os elementos do RDD que cumplen determinada condición.


In [6]:
# Exemplo: Elementos do RDD de enteiros que son pares.
print("Filtramos o RDD de enterios por pares: ",rdd_int.filter(lambda x: x % 2 == 0).collect())

# Exemplo: Elementos do RDD de palabras con lonxitude maior ou igual que 5.
print("Filtramos o RDD de palabras por lonxitude:: ",rdd_st.filter(lambda x: len(x) >= 5).collect())

Filtramos o RDD de enterios por pares:  [0, 2, 4, 6, 8]
Filtramos o RDD de palabras por lonxitude::  ['aplicado.', 'Curso', 'especialización', 'Inteligencia', 'Artificial']


### sortBy

Permite reordenar un **RDD** en función dun criterio definido polo programador, que pode especificarse mediante unha función *lambda*.  

Se se desexa realizar a ordenación en sentido inverso, é necesario indicalo explicitamente mediante o parámetro correspondente.


In [7]:
# Exemplo: RDD de palabras ordeado de forma ascendente.
print("RDD de palabras ordeado ascencentemente: ",rdd_st.sortBy(lambda x: len(x)).collect())

# Exemplo: RDD de palabras ordeado de forma descendente.
print("RDD de palabras ordeado descedentemente: ",rdd_st.sortBy(lambda x: len(x)*-1).collect())


RDD de palabras ordeado ascencentemente:  ['y', 'de', 'de', 'Big', 'Big', 'Data', 'Data', 'Curso', 'aplicado.', 'Artificial', 'Inteligencia', 'especialización']
RDD de palabras ordeado descedentemente:  ['especialización', 'Inteligencia', 'Artificial', 'aplicado.', 'Curso', 'Data', 'Data', 'Big', 'Big', 'de', 'de', 'y']


### randomSplit

Permite dividir un **RDD** en varios **RDDs**, devolvéndoos nun array, segundo un conxunto de **pesos** especificados polo programador.  

A división realízase de forma aleatoria, respectando aproximadamente as proporcións indicadas polos pesos.


In [8]:
# Exemplo 1: distribuimos o RDD de enteiros con pesos 40-60.
print("RDD de enteiros distribuido con pesos 40-60:")
for rdd in rdd_int.randomSplit([0.4, 0.6]):
    print(rdd.collect())

# Exemplo 2: distribuimos o RDD de strings con pesos 50-50.
print("RDD de strings distribuido con pesos 50-50:")
for rdd in rdd_st.randomSplit([0.5,0.5]):
    print(rdd.collect())


RDD de enteiros distribuido con pesos 40-60:
[0, 4, 6]
[1, 2, 3, 5, 7, 8, 9]
RDD de strings distribuido con pesos 50-50:
['Big', 'aplicado.', 'de', 'especialización', 'Artificial', 'y']
['Data', 'Curso', 'de', 'Inteligencia', 'Big', 'Data']


## Outras operacións
### foreachPartition

Permite especificar a función que se debe aplicar a **cada partición** do **RDD**, en lugar de aplicala elemento a elemento.

É especialmente útil para operacións con recursos externos (bases de datos, ficheiros, APIs), xa que reduce o número de conexións.


In [10]:
# Exemplo 1:

# Definición dunha función que se aplicará a cada partición do RDD.
# A función recibe un iterador cos elementos desa partición.
def f(iterator):
    # Percórrese cada elemento da partición
    for x in iterator:
        # Amosar o tipo do elemento
        print(type(x))
        # Amosar o valor do elemento
        print(x)

# Exemplo de uso de pipe():
# Envía o contido de cada partición a un comando do sistema operativo.
# Neste caso, "wc -l" conta o número de liñas.
# O resultado recóllese no driver mediante collect().
print(rdd_int_par.pipe("wc -l").collect())

# Aplicación da función f a cada partición do RDD.
# foreachPartition executa a función en cada partición,
# pero NON devolve resultados ao driver (retorna None).
print(rdd_int_par.foreachPartition(f))


['5', '5', '5', '5']
None


In [11]:
# Exemplo 2

# Crear un RDD a partir dunha colección local de enteiros.
# Spark repartirá automaticamente os elementos en varias particións.
rdd = sc.parallelize([1, 2, 3, 4, 5])

# O método glom() transforma cada partición nunha lista.
# Despois, collect() trae ao driver unha lista co contido
# de todas as particións.
partitions = rdd.glom().collect()

# Percorrer as particións obtidas e amosar o seu contido.
for i, partition in enumerate(partitions):
    print(f"Partition {i}: {partition}")



Partition 0: [1, 2]
Partition 1: [3, 4, 5]


### glom

A función **glom** converte cada partición dun **RDD** nun *array* (lista) de elementos.  
É unha función que pode resultar moi útil para fins didácticos ou de depuración.

Non obstante, debe empregarse con coidado, xa que se as particións son moi grandes,
os *arrays* resultantes poden consumir moita memoria e provocar erros.


In [13]:
# Exemplo: Obtemos unha lista de elementos por cada partición.
rdd_st_par.glom().collect()

[['Big', 'Data'],
 ['aplicado.', 'Curso'],
 ['de', 'especialización'],
 ['de', 'Inteligencia'],
 ['Artificial', 'y', 'Big', 'Data']]

### Almacenamento de RDDs en ficheiros

É posible almacenar os **RDDs** en ficheiros de texto. Para iso existen dous métodos principais:

- **saveAsTextFile**: almacena o RDD en ficheiros de texto.  
  É necesario especificar a ruta de saída e, opcionalmente, un códec de compresión.


In [14]:
# Exemplo: gardamos o RDD de palabras como arquivo de texto.
rdd_st.saveAsTextFile("/probas/texto.txt")

- **saveAsPickleFile**: permite almacenar un **RDD** serializando os seus elementos mediante *pickle* (Python).
  
  En contornos baseados en **HDFS**, este método xera ficheiros binarios que poden ser lidos posteriormente por Spark.  
  É útil para gardar obxectos Python complexos, pero **non é interoperable** con outras linguaxes nin ferramentas externas.


In [ ]:
# Exemplo: gardamos o RDD de palabras como arquivo serializado.
rdd_st.saveAsPickleFile("/probas/secuencia")

### Checkpointing
Permite almacenar estados intermedios para non ter que repetir toda a secuencia de operacións dende o principio.

In [ ]:
# Exemplo: Uso de checkpoint
sc.setCheckpointDir("/home/jovyan/checkpoints")
rdd_st.checkpoint()

### mapPartitions

O método **pipe** permite enviar, de forma semellante ás tubaxes dun *shell*, o contido dun **RDD** á entrada estándar dun comando do sistema operativo.  
Deste xeito, cada partición do RDD é procesada de maneira independente polo comando externo. A continuación móstrase un exemplo sinxelo empregando o comando **wc**.


In [19]:
# Exemplo: Enviamos as particións a unha tubería do shell.
rdd_st_par.pipe("wc -l").collect()

['2', '2', '2', '2', '4']

Ao observar o código anterior, destaca o feito de que **Spark executa as operacións a nivel de partición**, non elemento a elemento.  

En realidade, a función **map** é un *alias* que aplica unha transformación a cada fila utilizando internamente o mecanismo de **mapPartitions**. Isto significa que Spark procesa cada partición como un iterador, aplicando a función aos seus elementos.

Este comportamento fai posible executar operacións a nivel de fila empregando directamente **mapPartitions** e iteradores, o que permite:
- reducir o custo de inicialización de recursos,
- optimizar o acceso a sistemas externos,
- e mellorar o rendemento en determinados escenarios.*mapPartitions* a nivel de fila. Este hecho hace posible ejecutar la operación *map* a nivel de fila, empleando la función *mapPartitions* e iteradores.

In [17]:
# Exemplo: pasamos as particións a listas.
cosa = rdd_st_par.mapPartitions(lambda x: [list(x)])
print(type(cosa))
cosa.collect()


<class 'pyspark.rdd.PipelinedRDD'>


[['Big', 'Data'],
 ['aplicado.', 'Curso'],
 ['de', 'especialización'],
 ['de', 'Inteligencia'],
 ['Artificial', 'y', 'Big', 'Data']]